In [22]:
import matrix as m
import copy 

class BlockInterface:
    def __init__(self, is_collectable, is_movable, is_walkable, name, gameBoard, location = None):
        #document the block
        """
        Parameters
        ----------
        is_collectable : bool
            Whether the block can be collected by the player for example keys or diamonds.
        is_movable : bool
            Whether the block can be moved by the player or not.
        is_walkable : bool
            Whether the player can walk on the block or not.
        """
        
        self._is_collectable = is_collectable
        self._is_movable = is_movable
        self._is_walkable = is_walkable 
        self._name = name
        self._gameBoard = gameBoard
        self.location = location

    def walkOver(self):
        pass

    def unwalkOver(self):
        pass

    def move(self, from_block, to_block):
        pass

    def move_back(self, from_block, to_block):
        pass

    def isReachable(self):
        return False

    def set_location(self, location):
        self.location = location

    def get_location(self):
        return self.location

    def set_is_collectable(self, is_collectable):
        self._is_collectable = is_collectable
    
    def get_is_collectable(self):
        return self._is_collectable

    is_collectable = property(get_is_collectable, set_is_collectable)
    
    def set_is_movable(self, is_movable):
        self._is_movable = is_movable

    def get_is_movable(self):
        return self._is_movable

    is_movable = property(get_is_movable, set_is_movable)

    def set_is_walkable(self, is_walkable):
        self._is_walkable = is_walkable

    def get_is_walkable(self):
        return self._is_walkable

    is_walkable = property(get_is_walkable, set_is_walkable)

    def get_gameBoard(self):
        return self._gameBoard

    def set_gameBoard(self, gameBoard):
        self._gameBoard = gameBoard
    
    gameBoard = property(get_gameBoard, set_gameBoard)

    def get_name(self):
        return self._name

    def set_name(self, name):
        self._name = name

    name = property(get_name, set_name)
    
    

    def __str__(self):
        return self.name[0:1]

class Wall(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Wall", gameBoard)

class Floor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Floor", gameBoard)

class Destination(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Destination", gameBoard)


    def isReachable(self):
        #if there is no more diamonds to collect, the destination is reachable
        return super().get_gameBoard().remainingDiamonds == 0

class Key(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Key", gameBoard)

    def isReachable(self):
        return super().get_is_collectable() and super().get_gameBoard().has_key == False

    def move(self, from_block, to_block):
        #if the key is moved, it is no longer collectable
        #if to block is a child from Key object, walkOver is called
        if isinstance(to_block, Key):
            to_block.walkOver()

    def move_back(self, from_block, to_block):
        if isinstance(to_block, Key):
            to_block.unwalkOver()

    def walkOver(self):
        super().get_gameBoard().has_key = True
        super().set_is_collectable(False)

    def unwalkOver(self):
        super().get_gameBoard().has_key = False
        super().set_is_collectable(True)



class Diamond(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, True, "Diamond", gameBoard)
    
    def isReachable(self):
        return super().get_is_collectable()

    def move(self, from_block, to_block):
        #if to block is a child from Diamond object, walkOver is called
        if self == to_block and super():
            to_block.walkOver()
 

    def move_back(self, from_block, to_block):
        if self == to_block :
            to_block.unwalkOver()

    def walkOver(self):
        if(super().is_collectable):
            super().get_gameBoard().remainingDiamonds -= 1
            super().set_is_collectable(False)
            if super().get_gameBoard().remainingDiamonds == 0:
                super().get_gameBoard().openDestination()


    def unwalkOver(self):
        if(super().is_collectable == False):
            super().get_gameBoard().remainingDiamonds += 1
            super().set_is_collectable(True)
            super().get_gameBoard().closeDestination()


class KeyDoor(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(True, False, False, "KeyDoor", gameBoard)
        self.is_open = False
    
    def isReachable(self):
        return super().get_gameBoard().has_key == True and super().get_is_collectable() == True

    def move(self, from_block, to_block):
        #if to block is a child from KeyDoor object, walkOver is called
        if self == to_block:
            to_block.walkOver()

    def move_back(self, from_block, to_block):
        if self == to_block:
            to_block.unwalkOver()
    
    def walkOver(self):
        if self.is_open:
            return
        if super().get_gameBoard().has_key == True:
            super().get_gameBoard().has_key = False
            self.is_open = True
            super().set_is_collectable(False)
        else: #raise error if the player tries to walk over the door without a key
            raise Exception("You need a key to open this door")
    
    def unwalkOver(self):
        super().set_is_walkable(False)
        super().set_is_collectable(True)
        super().get_gameBoard().has_key = True
        self.is_open = False

    def get_is_walkable(self):
        return super().get_gameBoard().has_key

    def set_is_walkable(self, is_walkable):
        super().set_is_walkable(is_walkable)

    is_walkable = property(get_is_walkable, set_is_walkable)

class Spikes(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Spikes", gameBoard)

    def move(self, from_block, to_block):
        #if to block is a child from Spikes object, walkOver is called
        if self == from_block:
            from_block.walkOver()

    def move_back(self, from_block, to_block):
        if self == from_block:
            from_block.unwalkOver()

    def walkOver(self):
        super().set_is_walkable(False)

    def unwalkOver(self):
        super().set_is_walkable(True)

class Rock(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, True, True, "Rock", gameBoard)

    def move(self, from_block, to_block):
        #if to block is a child from Rock object, walkOver is called
        if isinstance(to_block, Rock):
            to_block.walkOver()

    def walkOver(self):
        pass

    def isReachable(self):
        return True

class Magma(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Magma", gameBoard)

    def walkOver(self):
        pass

class Hole(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, False, "Hole", gameBoard)

    def walkOver(self):
        pass

class Button(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Button", gameBoard)

    def walkOver(self):
        pass

class DoorButton(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "DoorButton", gameBoard)

    def walkOver(self):
        pass

class Character(BlockInterface):
    def __init__(self, gameBoard):
        super().__init__(False, False, True, "Character", gameBoard)

    def walkOver(self):
        pass



class BlockGenerator:
    """
    Class to generate all the objects based on a fixed map
    """
    def __init__(self):
        self.map = {
            'R' : Rock,
            'D' : Diamond,
            'P' : Character,
            'M' : Wall,
            'W' : Magma,
            'A' : Destination,
            'C' : Floor,
            'K' : KeyDoor,
            'L' : Key,
            'S' : Spikes,
            'H' : Hole,
            'B' : Button,
            'U' : DoorButton,
        }

    def generate(self, target, gameBoard):
        """
        returns a new Object based on the given char
        for example:
        'R' -> Rock
        'D' -> Diamond
        """
        return  self.map[target](gameBoard)


class GameBoard:
    def __init__(self):
        self.board = []
        self.has_key = False
        self.remainingDiamonds = 0
        self.characterLocation = (0,0)
        self.destinationLocation = (0,0)

    def initialize(self, gameMap, gameBoard):
        self.board = self.generateBoard(gameMap, gameBoard)
        self.generateNumberOfCollectables()

    def generateBoard(self, gameMap, gameBoard):
        """
        Generates the board based on the given map
        """
        board = []
        generator = BlockGenerator()
        for row in gameMap:
            board.append([generator.generate(target, gameBoard) for target in row])
        return board

    def generateNumberOfCollectables(self):
        """
        Iterates over the board and counts the number of diamonds and keys
        """
        for row in self.board:
            for block in row:
                if block.name == "Diamond":
                    self.remainingDiamonds += 1
                if block.name == "Character":
                    self.characterLocation = (self.board.index(row), row.index(block))
                if block.name == "Destination":
                    self.destinationLocation = (self.board.index(row), row.index(block))
    def openDestination(self):
        self.board[self.destinationLocation[0]][self.destinationLocation[1]].set_is_walkable(True)
    def closeDestination(self):
        self.board[self.destinationLocation[0]][self.destinationLocation[1]].set_is_walkable(True)


    def generateVisitedMatrix(self):
        """
        Generates a matrix of the same size as the board, but filled with False
        """
        return [[False for x in range(len(self.board[0]))] for y in range(len(self.board))]

    

    def getDirectionOfMovement(self, start_point, end_point):
        """
        Moves the character from the start_point to the end_point and returns the direction of the move
        (L, R, U, D)
        """
        direction = ""
        if start_point[0] == end_point[0]:
            if start_point[1] > end_point[1]:
                direction = "L"
            else:
                direction = "R"
        else:
            if start_point[0] > end_point[0]:
                direction = "U"
            else:
                direction = "D"

        return direction
        

    def isSolved(self):
        """
        Returns True if the map is solved
        """
        if self.characterLocation != self.destinationLocation:
            return False
        print("is solved")
        return True
            
    def getInstructionsLenght(self, instructions):
        """
        Returns the lenght of the instructions
        """
        lenght = 0
        for instruction in instructions:
            lenght += len(instruction)
        return lenght
    
    
    def getSpecialBlocksPath(self):
        """
        starting from the character location, get all the speacial blocks for example diamonds, keys, etc
        """
        #if the character is alrady at the destination, return an empty list
        if self.characterLocation == self.destinationLocation:
            return []


        #list of reachable blocks each element is dict with keys: block, location where location is a list of charters, each character is a location in the board
        reachableBlocks = {} 
        #matrix to keep track of visited blocks
        visited = self.generateVisitedMatrix()
        visited[self.characterLocation[0]][self.characterLocation[1]] = True
        #seacht the shortest path to each reachable location
        self.getSpecialBlocksRecursive(self.characterLocation, visited, reachableBlocks, [self.characterLocation])

        #convert the reachable blocks to a list of blocks
        reachableBlocksList = []
        for key in reachableBlocks:
            reachableBlocksList.append(reachableBlocks[key])
        
        #sort the list based on the lenght of the path
        reachableBlocksList.sort(key=lambda x: len(x))
        return reachableBlocksList


    def getSpecialBlocksRecursive(self, location, visited, reachableBlocks, path):
        """
        Recursive function to get the shortest path to all the reachable blocks
        """
        curr_row, curr_col = location

        #get the current block
        block = self.board[curr_row][curr_col]


        
        #if the current block is a special block, add it to the reachable blocks
        if block.isReachable():
            if block not in reachableBlocks:
                reachableBlocks[block] = copy.deepcopy(path)
            else:
                if len(path) < len(reachableBlocks[block]):
                    reachableBlocks[block] = copy.deepcopy(path)
            return
            
        #get the next blocks
        next_blocks = self.getNeighbors(location, visited)

        #for each next block, call the function recursively
        for next_block in next_blocks:
            path.append(next_block)
            visited[curr_row][curr_col] = True
            self.getSpecialBlocksRecursive(next_block, visited, reachableBlocks, path)
            path.pop()
            visited[curr_row][curr_col] = False

        
    def getNeighbors(self, location, visited):
        """
        returns a list of neighbors of the given location
        """
        neighbors = []
        curr_row, curr_col = location
        for next_row, next_col in [[curr_row + 1, curr_col], [curr_row - 1, curr_col], [curr_row, curr_col + 1], [curr_row, curr_col - 1]]:
            if not(next_row >= 0 and next_row < len(self.board) and next_col >= 0 and next_col < len(self.board[0])):
                continue
            if visited[next_row][next_col]:
                continue
            neighbors.append((next_row, next_col))

        #get the block of each neighbor and check if it is walkable
        neighbors = [neighbor for neighbor in neighbors if self.board[neighbor[0]][neighbor[1]].is_walkable]

        return neighbors
    
    def __str__(self) -> str:
        map_str = ""
        for row in self.board:
            for block in row:
                map_str += str(block)
            map_str += "\n"
        return map_str



    def moveBack(self, path):
        """
        Moves the character back to the given block reverting the changes made to the board
        """
        #revet the list of blocks 
        self.characterLocation = path[0]
        for i in range(len(path) - 1):
            start_point = path[i]
            end_point = path[i+1]
            from_block = self.board[start_point[0]][start_point[1]]
            to_block = self.board[end_point[0]][end_point[1]]
            self.board[start_point[0]][start_point[1]].move_back(from_block, to_block)
            self.board[end_point[0]][end_point[1]].move_back(from_block, to_block)

        
        
        pass

    def moveTo(self, path):
        """
        Moves the character to the given block

        parameters:
        block: the block to move to, its compossed by an array of points of the map where the character moves
        for example: [(3, 3), (2, 3), (1, 3), (0, 3)] it means that the character moves from (3, 3) to (0, 3)
        incluiding the block (0, 3) in the path
        """
        list_movements = []
        for i in range(len(path) - 1):
            start_point = path[i]
            end_point = path[i+1]
            moves = self.moveCharacter(start_point, end_point)
            list_movements.append(moves)
        return list_movements
        
    def moveCharacter(self, start_point, end_point):
        """
        Moves the character to the given point
        """
        #get the block at the point and call the walkOver function
        from_block = self.board[start_point[0]][start_point[1]]
        to_block = self.board[end_point[0]][end_point[1]]
        self.board[start_point[0]][start_point[1]].move(from_block, to_block)
        self.board[end_point[0]][end_point[1]].move(from_block, to_block)

        #update the character location
        self.characterLocation = end_point
        #return the direction of the move
        return self.getDirectionOfMovement(start_point, end_point)

    def getShortestSolutionPath(self):
        """
        Returns the shortest path to the solution
        This path should collect all the diamonds 
        """
        instructions = [] 
        #self.getShortestSolutionPathRecursive(instructions, shortestInstructions)
        self.getSinlgeSolutionPathRecursive(instructions)
        return instructions
        

    def getSinlgeSolutionPathRecursive(self, instructions):
        """
        Recursive function to get the shortest path to the solution
        """
        #if the character is alrady at the destination, return an empty list
        if self.isSolved():
            return True

        #get the reachable blocks
        reachableBlocks = self.getSpecialBlocksPath()

        #for each reachable block, move to it and call the function recursively
        for i, block in enumerate(reachableBlocks):
            print(i, self.remainingDiamonds, block)
            #move to the block
            list_movements = self.moveTo(block)
            instructions.extend(list_movements)
            #call the function recursively
            if self.getSinlgeSolutionPathRecursive(instructions):
                return True
            #move back
            self.moveBack(block)
            #remove the moves from the instructions
            for i in range(len(list_movements)):
                instructions.pop()

        return False


    def getShortestSolutionPathRecursive(self, instructions, shortestInstructions):
        """
        Recursive function to find the shortest path to the solution
        This path should collect all the diamonds 
        """
        #if the game is solved, check if the instructions are shorter than the shortest instructions
        if self.isSolved():
            if len(shortestInstructions) == 0:
                shortestInstructions.extend(instructions)
            elif self.getInstructionsLenght(instructions) < self.getInstructionsLenght(shortestInstructions):
                shortestInstructions.clear()
                shortestInstructions.extend(instructions)
            return

        #get the shortest path to all the reachable blocks
        reachableBlocks = self.getSpecialBlocksPath()
        print(reachableBlocks)

        #for each reachable block, move to it and call the function recursively
        for block in reachableBlocks:
            #move to the block
            moves = self.moveTo(block)
            #add the moves to the instructions
            instructions.extend(moves)
            #call the function recursively
            self.getShortestSolutionPathRecursive(instructions, shortestInstructions)
            #move back
            self.moveBack(block)
            #remove the moves from the instructions
            for i in range(len(moves)):
                instructions.pop()

    



In [23]:
#test for level 3
import matrix

Level3 = [['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'C', 'M', 'C', 'M', 'C', 'D', 'M'],
          ['M', 'P', 'C', 'M', 'M', 'C', 'K', 'C', 'A', 'M'],
          ['M', 'C', 'C', 'S', 'C', 'C', 'M', 'C', 'D', 'M'],
          ['M', 'M', 'S', 'M', 'K', 'M', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'C', 'M', 'C', 'C', 'M', 'D', 'D', 'M'],
          ['M', 'C', 'D', 'M', 'C', 'C', 'K', 'L', 'D', 'M'],
          ['M', 'M', 'C', 'M', 'C', 'C', 'M', 'D', 'D', 'M'],
          ['M', 'L', 'C', 'M', 'D', 'C', 'M', 'M', 'M', 'M'],
          ['M', 'C', 'M', 'M', 'C', 'C', 'M', 'C', 'C', 'M'],
          ['M', 'D', 'S', 'S', 'C', 'C', 'K', 'L', 'D', 'M'],
          ['M', 'C', 'M', 'M', 'M', 'L', 'M', 'C', 'C', 'M'],
          ['M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M']]

board = GameBoard()
board.initialize(Level3, board)
print(board.getShortestSolutionPath())


0 11 [(4, 1), (5, 1), (5, 2), (6, 2), (7, 2), (8, 2)]
0 10 [(8, 2), (9, 2), (10, 2), (10, 1)]
0 10 [(10, 1), (11, 1), (12, 1)]
0 9 [(12, 1), (12, 2), (12, 3), (12, 4), (11, 4), (10, 4)]
0 8 [(10, 4), (9, 4), (8, 4), (7, 4), (6, 4)]
0 8 [(6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 4), (12, 4), (12, 5), (13, 5)]
0 8 [(13, 5), (12, 5), (12, 6)]
0 8 [(12, 6), (12, 7)]
0 8 [(12, 7), (12, 8)]
0 7 [(12, 8), (12, 7), (12, 6), (12, 5), (11, 5), (10, 5), (9, 5), (8, 5), (8, 6)]


Exception: You need a key to open this door

In [144]:
#test for key doors
level = [
    ['P', 'D', 'L', 'K', 'A']
]

board = GameBoard()
board.initialize(level, board)


moves = board.getSpecialBlocksPath()
board.moveTo(moves[0])
moves2 = board.getSpecialBlocksPath()
board.moveTo(moves2[0])
moves3 = board.getSpecialBlocksPath()
board.moveTo(moves3[0])
moves4 = board.getSpecialBlocksPath()
board.moveTo(moves4[0])

board.moveBack(moves4[0])
print("has key", board.has_key == False)
board.moveBack(moves3[0])
print("has key", board.has_key == True)
print("Is walkable", board.board[0][3].is_walkable == True)
board.moveBack(moves2[0])
board.moveBack(moves[0])
print(board.remainingDiamonds == 1)



has key True
has key True
has key True
True


In [88]:
#test for go back spears FAILED, IMPROVE LATER
#test for move back
Level1 = [
          ["M","M","M","D"],
          ["A","D","S","D"],
          ["M","M","P","C"],
          ]

board = GameBoard()
board.initialize(Level1, board)
board.getSpecialBlocksPath()

[[(2, 2), (1, 2), (1, 3)], [(2, 2), (1, 2), (1, 1)]]

In [31]:
#test for move back
Level1 = [
          ["A","C","D"],
          ["C","C","D"],
          ["C","P","C"],
          ]

board = GameBoard()
board.initialize(Level1, board)

m1 = board.getSpecialBlocksPath()
board.moveTo(m1[0])
print("1 move")
print(board.remainingDiamonds)
print(board.characterLocation)
m2 = board.getSpecialBlocksPath()
board.moveTo(m2[0])
print("2 move")
print(board.remainingDiamonds)
print(board.characterLocation)
m3 = board.getSpecialBlocksPath()
board.moveTo(m3[0])
print("3 move")
print(board.remainingDiamonds)
print(board.characterLocation)
print("first move back")
board.moveBack(m3[0])
print(board.remainingDiamonds)
print(board.characterLocation)
print("second move back")
board.moveBack(m2[0])
print(board.remainingDiamonds)
print(board.characterLocation)
print("third move back")
board.moveBack(m3[0])
print(board.remainingDiamonds)
print(board.characterLocation)


1 move
1
(0, 2)
2 move
0
(1, 2)
3 move
0
(0, 0)
first move back
1
(1, 2)
second move back
2
(0, 2)
third move back
2
(1, 2)


In [90]:

# test for solution
import matrix

board = GameBoard()
board.initialize(matrix.Level3, board)
print(board.getShortestSolutionPath())
print(board.characterLocation)
print(board.isSolved())



0 11 [(4, 1), (5, 1), (5, 2), (6, 2), (7, 2), (8, 2)]
0 10 [(8, 2), (9, 2), (10, 2), (10, 1)]
0 10 [(10, 1), (11, 1), (12, 1)]
0 9 [(12, 1), (12, 2), (12, 3), (12, 4), (11, 4), (10, 4)]
[]
(4, 1)
False


In [30]:
#test for get neighbors
mapReachable = [
    ['C', 'M', 'P'],
    ['A', 'C', 'C'],
    ['C', 'C', 'C'],
]

visited = [
    [False, False, False],
    [False, False, True],
    [False, False, False],
]

board = GameBoard()
board.initialize(mapReachable, board)

board.getNeighbors([1, 1], visited)

[(2, 1)]

In [29]:
#test for solve the map
mapReachable = [
    ['A', 'S', 'D', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)
block = board.getSpecialBlocksPath()
board.moveTo(block[0])
print(board.characterLocation)
block = board.getSpecialBlocksPath()
board.moveTo(block[0])
print(board.characterLocation)
print(board.isSolved())



(0, 2)
(0, 0)
True


In [13]:
#test for movement 
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['C', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'S'],
    ['A', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)
block = board.getSpecialBlocksPath()
print(block)
list_movements = board.moveTo(block[1])
print("test list of movements correct")
print(list_movements == ['U', 'U', 'L', 'U', 'L', 'L']) 
print("test for player location")
print(board.characterLocation == (0,0))
print(board.board[2][3].is_walkable == False)



TypeError: getNeighbors() missing 1 required positional argument: 'visited'

In [14]:
#test walk over and undwalkover
gameBoard = GameBoard()

#TEST 1 for key block
keyBlock = BlockGenerator().generate('L', gameBoard)
print("Test for key block")
print(keyBlock.isReachable() == 1)
print(keyBlock.is_collectable == 1)
keyBlock.walkOver()
print(gameBoard.has_key == True) #despues de pisado debe tener una llave
print(keyBlock.isReachable() == 0) #no puede ser alcanzacble
print(keyBlock.is_collectable == 0) #no puede ser recogido
keyBlock.unwalkOver()
print(gameBoard.has_key == False) #despues de pisado no debe tener una llave
print(keyBlock.isReachable() == 1) #puede ser alcanzacble
print(keyBlock.is_collectable == 1) #puede ser recogido

#TEST 2 for diamond block
diamondBlock = BlockGenerator().generate('D', gameBoard)
print("Test for diamond block")
gameBoard.remainingDiamonds = 5
print(diamondBlock.isReachable() == 1)
print(diamondBlock.is_collectable == 1)
diamondBlock.walkOver()
print(diamondBlock.isReachable() == 0)
print(diamondBlock.is_collectable == 0)
print(gameBoard.remainingDiamonds == 4)
diamondBlock.unwalkOver()
print(diamondBlock.isReachable() == 1)
print(diamondBlock.is_collectable == 1)
print(gameBoard.remainingDiamonds == 5)

#test 3 for spears block
spearsBlock = BlockGenerator().generate('S', gameBoard)
print("Test for spears block")
print(spearsBlock.is_walkable == True)#puede ser pisado
spearsBlock.walkOver()
print(spearsBlock.is_walkable == False)#no puede ser pisado
spearsBlock.unwalkOver()
print(spearsBlock.is_walkable == True)#puede ser pisado






Test for key block
True
True
True
True
True
True
True
True
Test for diamond block
True
True
True
True
True
True
True
True
Test for spears block
True
True
True


In [8]:
# test for generator class
gameBoard = GameBoard()
generator = BlockGenerator()
print(isinstance(generator.generate("L", gameBoard), Key) == True)
print(isinstance(generator.generate("C", gameBoard), Floor) == True)
print(isinstance(generator.generate("A", gameBoard), Destination) == True)

True
True
True


In [ ]:
# test for direction of movement function
board = GameBoard()
print( board.getDirectionOfMovement((0, 0), (0, 1)) == "R")
print( board.getDirectionOfMovement((0, 0), (0, -1)) == "L")
print( board.getDirectionOfMovement((0, 0), (1, 0)) == "D")
print( board.getDirectionOfMovement((0, 0), (-1, 0)) == "U")

In [22]:
#test for is solved function
mapReachable = [
    ['C', 'C',],
    ['A', 'P',],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.destinationLocation == (1, 0))
print(board.characterLocation == (1, 1))
print(board.remainingDiamonds == 0)
print(board.isSolved() == False)
board.characterLocation = (1, 0)
print(board.isSolved() == True)
board.remainingDiamonds = 1
print(board.isSolved() == False)



True
True
True
True
True
True


In [44]:
#test of getting the reachable blocks
mapReachable = [
    ['L', 'C', 'C', 'D'],
    ['M', 'M', 'C', 'C'],
    ['C', 'M', 'C', 'C'],
    ['C', 'C', 'C', 'P'],
]

board = GameBoard()
board.initialize(mapReachable, board)

print(board.characterLocation == (3,3))
print(board.remainingDiamonds == 1)
print(board.getSpecialBlocksPath())

True
True
[{'block': <__main__.Diamond object at 0x0000024D068B4190>, 'path': [(3, 3), (2, 3), (1, 3), (0, 3)]}, {'block': <__main__.Key object at 0x0000024D068B4250>, 'path': [(3, 3), (2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)]}]


In [ ]:
board = GameBoard()
board.initialize(m.Level1, board)

print(board.getSpecialBlocksPath())

In [9]:
#test of generation a signle block
gameBoard = GameBoard()

#TEST 1 for exit block
exitBlock = BlockGenerator().generate('A', gameBoard)

#test to get if the exit is reachable when there are no diamonds
gameBoard.remainingDiamonds = 0
print("test 1: ", exitBlock.isReachable() == True)

#test to get if the exit is reachable when there are diamonds
gameBoard.remainingDiamonds = 1
print("test 2: ", exitBlock.isReachable() == False)

#-------------
#TEST 2 for Door Block
doorBlock = BlockGenerator().generate('K', gameBoard)

gameBoard.has_key = True
print("test 3: ", doorBlock.isReachable() == True)

gameBoard.has_key = False
print("test 4: ", doorBlock.isReachable() == False)






test 1:  True
test 2:  True
test 3:  True
test 4:  True
F
